In [1]:
import sys
sys.path.append('./')
sys.path.append('../')

from interventions import three_operands

from transformers import AutoTokenizer, AutoModelForCausalLM
from omegaconf import DictConfig, OmegaConf
from tqdm.notebook import tqdm
import torch
import random
import yaml
import numpy as np
import pickle
import seaborn as sns
import re
import copy

# from llm_transparency_tool.models.tlens_model import TransformerLensTransparentLlm
# from llm_transparency_tool.models.transparent_llm import ModelInfo
# from llm_transparency_tool.routes.graph import GraphBuilder, build_full_graph, build_paths_to_predictions
import networkx as nx

import matplotlib.pyplot as plt
import warnings

%matplotlib inline

In [2]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.set_grad_enabled(False)

In [3]:
model_name = 'EleutherAI/pythia-12b-deduped-v0'
model_name_lens = 'pythia-12b-deduped-v0'
# model_name = 'EleutherAI/pythia-6.9b-deduped-v0'
# model_name_lens = 'pythia-6.9b-deduped-v0'
# model_name = 'facebook/opt-1.3b'
# model_name_lens = 'opt-1.3b'
# model_name = 'openai-community/gpt2'
# model_name_lens = 'gpt2'
# model_name = 'facebook/opt-2.7b'
# model_name_lens = 'opt-2.7b'
# model_name = 'facebook/opt-6.7b'
# model_name_lens = 'opt-6.7b'
# model_name = 'facebook/opt-13b'
# model_name_lens = 'opt-13b'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model.device

device(type='cuda', index=0)

In [5]:
conf = OmegaConf.load('./conf/config.yaml')
conf.model = model_name
# conf.max_n = 9
conf.max_n = 20
# conf.n_shots = 1
# intervention_list = three_operands.get_arithmetic_data_three_operands(tokenizer, conf)
# intervention_list = pickle.load(open('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/intervention_1_shots_max_20_arabic_further_templates.pkl', 'rb'))
# intervention_list = pickle.load(open('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/information_flow_routes_results_opt_2_7b/base_data_arabic_opt_2_7.pkl', 'rb'))
# intervention_list = torch.load('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/information_flow_routes_results_opt_2_7b/base_data_arabic_opt_2_7.pkl')
# intervention_list = torch.load('/mnt/qb/work/eickhoff/esx208/arithmetic-lm/data/patching_results_opt_6_7b_old/intervention_list.pkl')
conf = OmegaConf.load('conf/config.yaml')
intervention_list = three_operands.get_arithmetic_data_three_operands(tokenizer, conf)

  0%|                                                                                                                                                                                                                                                                                       | 0/210 [00:00<?, ?it/s]

 46%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                                                  | 96/210 [00:00<00:00, 940.28it/s]

res base: 14
res base wo whitespace: 4
res base with prefix: a 14
res base tok: ['a', 'Ġ14']
operands_base:  ('3', '5', '6')
x_base_tok:  ['Ġ3']
y_base_tok:  ['Ġ5']
z_base_tok:  ['Ġ6']
base string tok list:  [21, 559, 577, 559, 721, 426, 1638, 15, 495, 559, 608, 559, 721, 426]
z_base_tok:  721
y_base_tok:  608
x_base_tok:  495
res base: 17
res base wo whitespace: 7
res base with prefix: a 17
res base tok: ['a', 'Ġ17']
operands_base:  ('3', '7', '7')
x_base_tok:  ['Ġ3']
y_base_tok:  ['Ġ7']
z_base_tok:  ['Ġ7']
base string tok list:  [20, 559, 374, 559, 854, 426, 2145, 15, 495, 559, 818, 559, 818, 426]
z_base_tok:  818
y_base_tok:  818
x_base_tok:  495
res base: 12
res base wo whitespace: 2
res base with prefix: a 12
res base tok: ['a', 'Ġ12']
operands_base:  ('8', '2', '2')
x_base_tok:  ['Ġ8']
y_base_tok:  ['Ġ2']
z_base_tok:  ['Ġ2']
base string tok list:  [21, 559, 374, 559, 854, 426, 1638, 15, 854, 559, 374, 559, 374, 426]
z_base_tok:  374
y_base_tok:  374
x_base_tok:  854
res base: 14


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 210/210 [00:00<00:00, 237.67it/s]

res base: 12
res base wo whitespace: 2
res base with prefix: a 12
res base tok: ['a', 'Ġ12']
operands_base:  ('3', '2', '2')
x_base_tok:  ['Ġ3']
y_base_tok:  ['Ġ2']
z_base_tok:  ['Ġ2']
base string tok list:  [20, 475, 374, 475, 495, 426, 1283, 15, 495, 475, 374, 475, 374, 426]
z_base_tok:  374
y_base_tok:  374
x_base_tok:  495
res base: 16
res base wo whitespace: 6
res base with prefix: a 16
res base tok: ['a', 'Ġ16']
operands_base:  ('2', '2', '4')
x_base_tok:  ['Ġ2']
y_base_tok:  ['Ġ2']
z_base_tok:  ['Ġ4']
base string tok list:  [19, 475, 495, 475, 495, 426, 1283, 15, 374, 475, 374, 475, 577, 426]
z_base_tok:  577
y_base_tok:  374
x_base_tok:  374
res base: 18
res base wo whitespace: 8
res base with prefix: a 18
res base tok: ['a', 'Ġ18']
operands_base:  ('2', '3', '3')
x_base_tok:  ['Ġ2']
y_base_tok:  ['Ġ3']
z_base_tok:  ['Ġ3']
base string tok list:  [19, 475, 374, 475, 495, 426, 1249, 15, 374, 475, 495, 475, 495, 426]
z_base_tok:  495
y_base_tok:  495
x_base_tok:  374
res base: 12


In [23]:
def calc_accuracy(intervention_list, model):
    accuracy_base = []
    accuracy_alt = []

    accuracy_base_tok = []
    accuracy_alt_tok = []

    for item in tqdm(intervention_list):
        # input_id_base = item.base_string_tok.to(model.device)
        # input_id_base = torch.tensor([tokenizer.bos_token_id] + item.base_string_tok_list).unsqueeze(0).to(model.device)
        
        # For other models
        # input_id_base = torch.tensor(tokenizer.encode(' ' + item.few_shots + item.base_string)).unsqueeze(0).to(model.device)
        # input_id_alt = item.alt_string_tok.to(model.device)
        
        # For other models and for testing the results without padding
        # input_id_base = torch.tensor(tokenizer.encode(' ' + item.few_shots.lstrip() + item.base_string)).unsqueeze(0).to(model.device)
        # input_id_alt = torch.tensor(tokenizer.encode(' ' + item.base_string.lstrip())).unsqueeze(0).to(model.device)
        
        # For pythia and for testing the results without padding
        input_id_base = torch.tensor([tokenizer.bos_token_id] + tokenizer.encode(' ' + item.few_shots + item.base_string)).unsqueeze(0).to(model.device)
        input_id_alt = torch.tensor([tokenizer.bos_token_id] + tokenizer.encode(' ' + item.base_string)).unsqueeze(0).to(model.device)

        output_base = model.generate(input_id_base, max_new_tokens=1, do_sample=False)[0, -1].cpu().numpy()
        output_alt = model.generate(input_id_alt, max_new_tokens=1, do_sample=False)[0, -1].cpu().numpy()
        output_base_str = tokenizer.decode(output_base)
        output_alt_str = tokenizer.decode(output_alt)
        correct_output_tok = item.res_base_tok[0]
        correct_output_str = item.res_base_string

        # print(output_base, output_alt, correct_output_tok)
        # correct_output_str = int(item.res_base_string)
        print('With icd: ', output_base_str, 'Without icd: ', output_alt_str, 'Correct: ', correct_output_str)
        print(tokenizer.decode(input_id_base[0]))
        print(tokenizer.decode(input_id_alt[0]))
        print('#' * 10)

        try:
            accuracy_base.append(int(output_base_str) == int(correct_output_str))
        except:
            accuracy_base.append(0)
        
        try:
            accuracy_alt.append(int(output_alt_str) == int(correct_output_str))
        except:
            accuracy_alt.append(0)
        
        accuracy_base_tok.append(output_base == correct_output_tok)
        accuracy_alt_tok.append(output_alt == correct_output_tok)

    return accuracy_base, accuracy_alt, accuracy_base_tok, accuracy_alt_tok

accuracy_base, accuracy_alt, accuracy_base_tok, accuracy_alt_tok = calc_accuracy(intervention_list, model)

  0%|          | 0/210 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   14 Without icd:   14 Correct:  14
<|endoftext|> 4 + 4 + 6 = 14. 3 + 5 + 6 =
<|endoftext|> 3 + 5 + 6 =
##########
With icd:   15 Without icd:   20 Correct:  17
<|endoftext|> 3 + 2 + 8 = 13. 3 + 7 + 7 =
<|endoftext|> 3 + 7 + 7 =
##########
With icd:   12 Without icd:   12 Correct:  12
<|endoftext|> 4 + 2 + 8 = 14. 8 + 2 + 2 =
<|endoftext|> 8 + 2 + 2 =
##########
With icd:   14 Without icd:   14 Correct:  14
<|endoftext|> 10 + 2 + 3 = 15. 2 + 6 + 6 =
<|endoftext|> 2 + 6 + 6 =
##########
With icd:   19 Without icd:   19 Correct:  19
<|endoftext|> 5 + 4 + 2 = 11. 8 + 6 + 5 =
<|endoftext|> 8 + 6 + 5 =
##########
With icd:   11 Without icd:   11 Correct:  11
<|endoftext|> 9 + 2 + 2 = 13. 6 + 2 + 3 =
<|endoftext|> 6 + 2 + 3 =
##########
With icd:   17 Without icd:   17 Correct:  17
<|endoftext|> 4 + 6 + 8 = 18. 2 + 9 + 6 =
<|endoftext|> 2 + 9 + 6 =
##########


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

...

With icd:   16 Without icd:   - Correct:  4
<|endoftext|> The result of ( 16 - 13 ) * 2 = 6. The result of ( 10 - 9 ) * 4 =
<|endoftext|> The result of ( 10 - 9 ) * 4 =
##########
With icd:   2 Without icd:   - Correct:  2
<|endoftext|> The result of ( 9 - 8 ) * 9 = 9. The result of ( 13 - 12 ) * 2 =
<|endoftext|> The result of ( 13 - 12 ) * 2 =
##########
With icd:   5 Without icd:   - Correct:  10
<|endoftext|> The result of ( 10 - 9 ) * 18 = 18. The result of ( 19 - 17 ) * 5 =
<|endoftext|> The result of ( 19 - 17 ) * 5 =
##########
With icd:   6 Without icd:   - Correct:  6
<|endoftext|> The result of ( 8 - 6 ) * 3 = 6. The result of ( 8 - 7 ) * 6 =
<|endoftext|> The result of ( 8 - 7 ) * 6 =
##########
With icd:   18 Without icd:   - Correct:  18
<|endoftext|> The result of ( 16 - 14 ) * 7 = 14. The result of ( 3 - 2 ) * 18 =
<|endoftext|> The result of ( 3 - 2 ) * 18 =
##########
With icd:   15 Without icd:   - Correct:  12
<|endoftext|> The result of ( 11 - 8 ) * 5 = 15. The res

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   6 Without icd:   - Correct:  12
<|endoftext|> The result of ( 4 - 2 ) * 3 = 6. The result of ( 13 - 7 ) * 2 =
<|endoftext|> The result of ( 13 - 7 ) * 2 =
##########
With icd:   2 Without icd:   - Correct:  6
<|endoftext|> The result of ( 13 - 12 ) * 8 = 8. The result of ( 15 - 12 ) * 2 =
<|endoftext|> The result of ( 15 - 12 ) * 2 =
##########
With icd:   90 Without icd:   - Correct:  6
<|endoftext|> The result of ( 6 - 3 ) * 3 = 9. The result of ( 15 - 14 ) * 6 =
<|endoftext|> The result of ( 15 - 14 ) * 6 =
##########
With icd:   14 Without icd:   - Correct:  14
<|endoftext|> The result of ( 11 - 10 ) * 15 = 15. The result of ( 11 - 10 ) * 14 =
<|endoftext|> The result of ( 11 - 10 ) * 14 =
##########
With icd:   3 Without icd:   - Correct:  12
<|endoftext|> The result of ( 6 - 5 ) * 2 = 2. The result of ( 19 - 15 ) * 3 =
<|endoftext|> The result of ( 19 - 15 ) * 3 =
##########
With icd:   2 Without icd:   ( Correct:  16
<|endoftext|> The result of ( 9 - 8 ) * 12 = 12. T

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad

With icd:   16 Without icd:   - Correct:  16
<|endoftext|> The result of ( 12 - 10 ) * 2 = 4. The result of ( 11 - 10 ) * 16 =
<|endoftext|> The result of ( 11 - 10 ) * 16 =
##########
With icd:   2 Without icd:   - Correct:  4
<|endoftext|> The result of ( 7 - 6 ) * 11 = 11. The result of ( 19 - 17 ) * 2 =
<|endoftext|> The result of ( 19 - 17 ) * 2 =
##########
With icd:   4 Without icd:   - Correct:  6
<|endoftext|> The result of ( 15 - 13 ) * 2 = 4. The result of ( 9 - 6 ) * 2 =
<|endoftext|> The result of ( 9 - 6 ) * 2 =
##########
With icd:   3 Without icd:   - Correct:  12
<|endoftext|> The result of ( 16 - 14 ) * 9 = 18. The result of ( 13 - 9 ) * 3 =
<|endoftext|> The result of ( 13 - 9 ) * 3 =
##########
With icd:   16 Without icd:   ( Correct:  2
<|endoftext|> The result of ( 18 - 10 ) * 2 = 16. The result of ( 9 - 8 ) * 2 =
<|endoftext|> The result of ( 9 - 8 ) * 2 =
##########
With icd:   15 Without icd:   - Correct:  18
<|endoftext|> The result of ( 8 - 7 ) * 10 = 10. The

In [24]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok))

Accuracy with icd, numbers equality 0.6047619047619047
Accuracy without icd, numbers equality 0.5666666666666667
Accuracy with icd, tokens equality 0.6047619047619047
Accuracy without icd, tokens equality 0.5666666666666667


In [25]:
def break_down_accuracy(intervention_list, accuracy):
    number_pattern = r"(?:\d+|one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty)"

    templates = [
        re.compile(rf"The result of {number_pattern} \* {number_pattern} \* {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of {number_pattern} times {number_pattern} times {number_pattern} = {number_pattern}\."),
        re.compile(rf"{number_pattern} \* {number_pattern} \* {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of {number_pattern} \+ {number_pattern} \+ {number_pattern} = {number_pattern}\."),
        re.compile(rf"{number_pattern} \+ {number_pattern} \+ {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of \({number_pattern} - {number_pattern}\) \* {number_pattern} = {number_pattern}\."),
        re.compile(rf"The result of {number_pattern} minus {number_pattern} minus {number_pattern} is {number_pattern}\.")
    ]

    accuracies_per_template = [[], [], [], [], [], [], []]

    for i in range(len(accuracy)):
        for j, template in enumerate(templates):
            if template.match(intervention_list[i].few_shots):
                accuracies_per_template[j].append(accuracy_base[i])
    
    return [np.sum(item) if item else 0 for item in accuracies_per_template]

In [26]:
break_down_accuracy(intervention_list, accuracy_base)

[np.int64(28),
 np.int64(24),
 np.int64(14),
 np.int64(25),
 np.int64(18),
 0,
 np.int64(6)]

In [27]:
np.array(break_down_accuracy(intervention_list, accuracy_base)) / 30

array([0.93333333, 0.8       , 0.46666667, 0.83333333, 0.6       ,
       0.        , 0.2       ])

Let's calculate scores on the dataset with word representation

In [28]:
if 'facebook/opt' in model_name:
    SYMBOLS = ["alpha", "beta", "lambda", "delta", "omega", "mu", "nu", "pi", "chi", "number", "res", "x", "y", "z", "a", "b", "c"]
    WORD_NUMBERS = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine",
        "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty"
    ]
elif 'EleutherAI/pythia' in model_name:
    SYMBOLS = ["alpha", "beta", "gamma", "delta", "mu", "nu", "chi", "theta", "sigma", "number", "res", "x", "y", "z", "a", "b", "c"]
    WORD_NUMBERS = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine",
        "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty"
    ]
    
for item in SYMBOLS:
    print(tokenizer.tokenize(' ' + item))
for item in WORD_NUMBERS:
    print(tokenizer.tokenize(' ' + item))

['Ġalpha']
['Ġbeta']
['Ġgamma']
['Ġdelta']
['Ġmu']
['Ġnu']
['Ġchi']
['Ġtheta']
['Ġsigma']
['Ġnumber']
['Ġres']
['Ġx']
['Ġy']
['Ġz']
['Ġa']
['Ġb']
['Ġc']
['Ġzero']
['Ġone']
['Ġtwo']
['Ġthree']
['Ġfour']
['Ġfive']
['Ġsix']
['Ġseven']
['Ġeight']
['Ġnine']
['Ġten']
['Ġeleven']
['Ġtwelve']
['Ġthirteen']
['Ġfourteen']
['Ġfifteen']
['Ġsixteen']
['Ġseventeen']
['Ġeighteen']
['Ġnineteen']
['Ġtwenty']


In [29]:
intervention_list[0].__dict__

{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': '3 5 6',
 'operands_base': '3 5 6',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [1638],
 'res_base_tok': [1638],
 'res_string': None,
 'res_base_string': '14',
 'res_alt_string': '14',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': '3 + 5 + 6 =',
 'alt_string': '3 + 5 + 6 =',
 'few_shots': '4 + 4 + 6 = 14. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-12b-deduped-v0', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False

In [ ]:
def replace_numbers_with(intervention_list, conf, operands=False, result_few_shot=False, result_prompt=False, few_shot=False, prompt=False, replacement = 'symbols'):
    counterfactual_intervention_list = []
    for intervention in tqdm(intervention_list):
        # Create copy of intervention
        new_intervention = copy.deepcopy(intervention)
        
        def replace(text, operands, result):
            # Get few shot example and tokenize it
            text_tokenized = tokenizer.encode(text, add_special_tokens=False)        
            
            # Get positions of numbers
            number_positions = []
            for pos, item in enumerate(text_tokenized):
                if tokenizer.decode(item).strip().isnumeric():
                    number_positions.append(pos)
            
            number_positions_selected = []
            if operands:
                number_positions_selected += number_positions[:-1]
            if result:
                number_positions_selected += number_positions[-1:]
            
            # For all position in number positions randomly choose symbol to replace and 
            for pos in number_positions_selected:
                if replacement == 'symbols':
                    result_str = ' ' + random.choice(SYMBOLS)
                    new_result_enc = tokenizer.encode(result_str, add_special_tokens=False)[0]
                    text_tokenized[pos] = new_result_enc
                elif replacement == 'words':
                    number = int(tokenizer.decode(text_tokenized[pos]).strip())
                    num_word = ' ' + WORD_NUMBERS[number]
                    num_word_enc = tokenizer.encode(num_word, add_special_tokens=False)[0]
                    text_tokenized[pos] = num_word_enc
                elif replacement == 'random':
                    if result and not operands:
                        result_int = int(tokenizer.decode(text_tokenized[pos]).strip())
                        new_result = random.randint(1, conf.max_n)
                        while new_result == result_int:
                            new_result = random.randint(1, conf.max_n)
                        new_result_enc = tokenizer.encode(' ' + str(new_result), add_special_tokens=False)[0]
                        text_tokenized[pos] = new_result_enc
                    else:
                        raise NotImplementedError('Only supported for altering the result')
                else:
                    raise NotImplementedError(replacement + ' ' + 'not implemented!')
            
            return text_tokenized, tokenizer.decode(text_tokenized)
                
        
        if few_shot:
            few_shot_tokenized, few_shot_text = replace(intervention.few_shots, operands=operands, result=result_few_shot)
            new_intervention.few_shots = few_shot_text
    
        if prompt:
            prompt_tokenized, prompt_text = replace(intervention.base_string, operands=operands, result=operands)
            new_intervention.base_string_tok = prompt_tokenized
            new_intervention.base_string = prompt_text
        if result_prompt:
            result_tokenized, result_text = replace(intervention.res_base_string, operands=False, result=True)
            new_intervention.res_base_tok = result_tokenized
            new_intervention.res_base_string = result_text
                
        counterfactual_intervention_list.append(new_intervention)
    return counterfactual_intervention_list

In [67]:
intervention_list_texts = replace_numbers_with(intervention_list, conf, operands=True, result_few_shot=True, result_prompt=True, few_shot=True, prompt=True, replacement='words')

  0%|          | 0/210 [00:00<?, ?it/s]

In [69]:
intervention_list_texts[0].__dict__

{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': '3 5 6',
 'operands_base': '3 5 6',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [1638],
 'res_base_tok': [25963],
 'res_string': None,
 'res_base_string': ' fourteen',
 'res_alt_string': '14',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': ' three + five + six =',
 'alt_string': '3 + 5 + 6 =',
 'few_shots': ' four + four + six = fourteen. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-12b-deduped-v0', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, c

In [70]:
accuracy_base_words, accuracy_alt_words, accuracy_base_tok_words, accuracy_alt_tok_words = calc_accuracy(intervention_list_texts, model)

  0%|          | 0/210 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   fifteen Without icd:   _____ Correct:   fourteen
<|endoftext|>  four + four + six = fourteen.  three + five + six =
<|endoftext|>  three + five + six =
##########
With icd:   fifteen Without icd:   twenty Correct:   seventeen
<|endoftext|>  three + two + eight = thirteen.  three + seven + seven =
<|endoftext|>  three + seven + seven =
##########
With icd:   ten Without icd:   ten Correct:   twelve
<|endoftext|>  four + two + eight = fourteen.  eight + two + two =
<|endoftext|>  eight + two + two =
##########
With icd:   twelve Without icd:   ten Correct:   fourteen
<|endoftext|>  ten + two + three = fifteen.  two + six + six =
<|endoftext|>  two + six + six =
##########
With icd:   twenty Without icd:   twenty Correct:   nineteen
<|endoftext|>  five + four + two = eleven.  eight + six + five =
<|endoftext|>  eight + six + five =
##########
With icd:   eleven Without icd:  
 Correct:   eleven
<|endoftext|>  nine + two + two = thirteen.  six + two + three =
<|endoftext|>  six

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   eighteen Without icd:   ? Correct:   eighteen
<|endoftext|>  three + six + two = eleven.  five + ten + three =
<|endoftext|>  five + ten + three =
##########
With icd:   twenty Without icd:   twenty Correct:   eighteen
<|endoftext|>  three + three + seven = thirteen.  four + nine + five =
<|endoftext|>  four + nine + five =
##########
With icd:   sixteen Without icd:   _____ Correct:   fourteen
<|endoftext|>  five + five + seven = seventeen.  two + nine + three =
<|endoftext|>  two + nine + three =
##########
With icd:   seventeen Without icd:   seventeen Correct:   fifteen
<|endoftext|>  nine + two + seven = eighteen.  two + ten + three =
<|endoftext|>  two + ten + three =
##########
With icd:   twenty Without icd:   twenty Correct:   eighteen
<|endoftext|>  seven + two + six = fifteen.  seven + four + seven =
<|endoftext|>  seven + four + seven =
##########
With icd:   twelve Without icd:   ten Correct:   fourteen
<|endoftext|>  four + eleven + two = seventeen.  two + fou

...

With icd:   two Without icd:   - Correct:   two
<|endoftext|> The result of ( nine - eight ) * nine = nine. The result of ( thirteen - twelve ) * two =
<|endoftext|> The result of ( thirteen - twelve ) * two =
##########
With icd:   five Without icd:   - Correct:   ten
<|endoftext|> The result of ( ten - nine ) * eighteen = eighteen. The result of ( nineteen - seventeen ) * five =
<|endoftext|> The result of ( nineteen - seventeen ) * five =
##########
With icd:   six Without icd:   - Correct:   six
<|endoftext|> The result of ( eight - six ) * three = six. The result of ( eight - seven ) * six =
<|endoftext|> The result of ( eight - seven ) * six =
##########
With icd:   eighteen Without icd:   ( Correct:   eighteen
<|endoftext|> The result of ( sixteen - fourteen ) * seven = fourteen. The result of ( three - two ) * eighteen =
<|endoftext|> The result of ( three - two ) * eighteen =
##########
With icd:   thirty Without icd:   - Correct:   twelve
<|endoftext|> The result of ( eleven 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   two Without icd:   - Correct:   six
<|endoftext|> The result of ( thirteen - twelve ) * eight = eight. The result of ( fifteen - twelve ) * two =
<|endoftext|> The result of ( fifteen - twelve ) * two =
##########
With icd:   twenty Without icd:   - Correct:   six
<|endoftext|> The result of ( six - three ) * three = nine. The result of ( fifteen - fourteen ) * six =
<|endoftext|> The result of ( fifteen - fourteen ) * six =
##########
With icd:   fourteen Without icd:   ( Correct:   fourteen
<|endoftext|> The result of ( eleven - ten ) * fifteen = fifteen. The result of ( eleven - ten ) * fourteen =
<|endoftext|> The result of ( eleven - ten ) * fourteen =
##########
With icd:   three Without icd:   - Correct:   twelve
<|endoftext|> The result of ( six - five ) * two = two. The result of ( nineteen - fifteen ) * three =
<|endoftext|> The result of ( nineteen - fifteen ) * three =
##########
With icd:   two Without icd:   eleven Correct:   sixteen
<|endoftext|> The result o

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   two Without icd:   - Correct:   four
<|endoftext|> The result of ( seven - six ) * eleven = eleven. The result of ( nineteen - seventeen ) * two =
<|endoftext|> The result of ( nineteen - seventeen ) * two =
##########
With icd:   six Without icd:   - Correct:   six
<|endoftext|> The result of ( fifteen - thirteen ) * two = four. The result of ( nine - six ) * two =
<|endoftext|> The result of ( nine - six ) * two =
##########
With icd:   twenty Without icd:   - Correct:   twelve
<|endoftext|> The result of ( sixteen - fourteen ) * nine = eighteen. The result of ( thirteen - nine ) * three =
<|endoftext|> The result of ( thirteen - nine ) * three =
##########
With icd:   four Without icd:   - Correct:   two
<|endoftext|> The result of ( eighteen - ten ) * two = sixteen. The result of ( nine - eight ) * two =
<|endoftext|> The result of ( nine - eight ) * two =
##########
With icd:   three Without icd:   - Correct:   eighteen
<|endoftext|> The result of ( eight - seven ) * t

In [73]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_words))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_words))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_words))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_words))

Accuracy with icd, numbers equality 0.0
Accuracy without icd, numbers equality 0.0
Accuracy with icd, tokens equality 0.5
Accuracy without icd, tokens equality 0.11428571428571428


In [74]:
break_down_accuracy(intervention_list_texts, accuracy_base_tok_words)

[np.int64(28), np.int64(24), 0, np.int64(25), 0, 0, np.int64(6)]

In [75]:
(np.array(break_down_accuracy(intervention_list, accuracy_base)) + np.array(break_down_accuracy(intervention_list_texts, accuracy_base_tok_words))) / 60

array([0.93333333, 0.8       , 0.23333333, 0.83333333, 0.3       ,
       0.        , 0.2       ])

In [76]:
print('Accuracy with icd, all', np.mean(accuracy_base_words + accuracy_base))
print('Accuracy without icd, all', np.mean(accuracy_alt_words + accuracy_alt))

print('Accuracy with icd, all', np.mean(accuracy_base_tok_words + accuracy_base_tok))
print('Accuracy without icd, all', np.mean(accuracy_alt_tok_words + accuracy_alt_tok))

Accuracy with icd, all 0.30238095238095236
Accuracy without icd, all 0.2833333333333333
Accuracy with icd, all 0.5523809523809524
Accuracy without icd, all 0.3404761904761905


Let's calculate accuracy with result tokens repaced with numbers but answer tokens left as is and also results with result tokens replaced with numbers and the answer token also replaced with number

In [77]:
intervention_list_only_results = replace_numbers_with(intervention_list, conf, operands=False, result_few_shot=True, result_prompt=True, few_shot=True, prompt=True, replacement='words')

  0%|          | 0/210 [00:00<?, ?it/s]

In [78]:
intervention_list_only_results[0].__dict__

{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': '3 5 6',
 'operands_base': '3 5 6',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [1638],
 'res_base_tok': [25963],
 'res_string': None,
 'res_base_string': ' fourteen',
 'res_alt_string': '14',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': '3 + 5 + 6 =',
 'alt_string': '3 + 5 + 6 =',
 'few_shots': '4 + 4 + 6 = fourteen. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-12b-deduped-v0', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenizatio

In [80]:
intervention_list_only_results_words = replace_numbers_with(intervention_list, conf, operands=True, result_few_shot=False, result_prompt=False, few_shot=True, prompt=True, replacement='words')

  0%|          | 0/210 [00:00<?, ?it/s]

In [81]:
intervention_list_only_results_words[0].__dict__

{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': '3 5 6',
 'operands_base': '3 5 6',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [1638],
 'res_base_tok': [1638],
 'res_string': None,
 'res_base_string': '14',
 'res_alt_string': '14',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': ' three + five + six =',
 'alt_string': '3 + 5 + 6 =',
 'few_shots': ' four + four + six = 14. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-12b-deduped-v0', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokeni

In [82]:
accuracy_base_only_results, accuracy_alt_only_results, accuracy_base_tok_only_results, accuracy_alt_tok_only_results = calc_accuracy(intervention_list_only_results, model)

  0%|          | 0/210 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   fourteen Without icd:   14 Correct:   fourteen
<|endoftext|> 4 + 4 + 6 = fourteen. 3 + 5 + 6 =
<|endoftext|> 3 + 5 + 6 =
##########
With icd:   eighteen Without icd:   20 Correct:   seventeen
<|endoftext|> 3 + 2 + 8 = thirteen. 3 + 7 + 7 =
<|endoftext|> 3 + 7 + 7 =
##########
With icd:   14 Without icd:   12 Correct:   twelve
<|endoftext|> 4 + 2 + 8 = fourteen. 8 + 2 + 2 =
<|endoftext|> 8 + 2 + 2 =
##########
With icd:   14 Without icd:   14 Correct:   fourteen
<|endoftext|> 10 + 2 + 3 = fifteen. 2 + 6 + 6 =
<|endoftext|> 2 + 6 + 6 =
##########
With icd:   nineteen Without icd:   19 Correct:   nineteen
<|endoftext|> 5 + 4 + 2 = eleven. 8 + 6 + 5 =
<|endoftext|> 8 + 6 + 5 =
##########
With icd:   thirteen Without icd:   11 Correct:   eleven
<|endoftext|> 9 + 2 + 2 = thirteen. 6 + 2 + 3 =
<|endoftext|> 6 + 2 + 3 =
##########
With icd:   seventeen Without icd:   17 Correct:   seventeen
<|endoftext|> 4 + 6 + 8 = eighteen. 2 + 9 + 6 =
<|endoftext|> 2 + 9 + 6 =
##########


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   eighteen Without icd:   18 Correct:   eighteen
<|endoftext|> 3 + 6 + 2 = eleven. 5 + 10 + 3 =
<|endoftext|> 5 + 10 + 3 =
##########
With icd:   nineteen Without icd:   18 Correct:   eighteen
<|endoftext|> 3 + 3 + 7 = thirteen. 4 + 9 + 5 =
<|endoftext|> 4 + 9 + 5 =
##########
With icd:   seventeen Without icd:   14 Correct:   fourteen
<|endoftext|> 5 + 5 + 7 = seventeen. 2 + 9 + 3 =
<|endoftext|> 2 + 9 + 3 =
##########
With icd:   17 Without icd:   15 Correct:   fifteen
<|endoftext|> 9 + 2 + 7 = eighteen. 2 + 10 + 3 =
<|endoftext|> 2 + 10 + 3 =
##########
With icd:   eighteen Without icd:   20 Correct:   eighteen
<|endoftext|> 7 + 2 + 6 = fifteen. 7 + 4 + 7 =
<|endoftext|> 7 + 4 + 7 =
##########
With icd:   sixteen Without icd:   12 Correct:   fourteen
<|endoftext|> 4 + 11 + 2 = seventeen. 2 + 4 + 8 =
<|endoftext|> 2 + 4 + 8 =
##########
With icd:   twelve Without icd:   12 Correct:   twelve
<|endoftext|> 12 + 2 + 4 = eighteen. 5 + 5 + 2 =
<|endoftext|> 5 + 5 + 2 =
#########

...

With icd:   twenty Without icd:   - Correct:   four
<|endoftext|> The result of ( 16 - 13 ) * 2 = six. The result of ( 10 - 9 ) * 4 =
<|endoftext|> The result of ( 10 - 9 ) * 4 =
##########
With icd:   two Without icd:   - Correct:   two
<|endoftext|> The result of ( 9 - 8 ) * 9 = nine. The result of ( 13 - 12 ) * 2 =
<|endoftext|> The result of ( 13 - 12 ) * 2 =
##########
With icd:   five Without icd:   - Correct:   ten
<|endoftext|> The result of ( 10 - 9 ) * 18 = eighteen. The result of ( 19 - 17 ) * 5 =
<|endoftext|> The result of ( 19 - 17 ) * 5 =
##########
With icd:   six Without icd:   - Correct:   six
<|endoftext|> The result of ( 8 - 6 ) * 3 = six. The result of ( 8 - 7 ) * 6 =
<|endoftext|> The result of ( 8 - 7 ) * 6 =
##########
With icd:   eighteen Without icd:   - Correct:   eighteen
<|endoftext|> The result of ( 16 - 14 ) * 7 = fourteen. The result of ( 3 - 2 ) * 18 =
<|endoftext|> The result of ( 3 - 2 ) * 18 =
##########
With icd:   thirty Without icd:   - Correct:  

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   twenty Without icd:   - Correct:   twelve
<|endoftext|> The result of ( 4 - 2 ) * 3 = six. The result of ( 13 - 7 ) * 2 =
<|endoftext|> The result of ( 13 - 7 ) * 2 =
##########
With icd:   two Without icd:   - Correct:   six
<|endoftext|> The result of ( 13 - 12 ) * 8 = eight. The result of ( 15 - 12 ) * 2 =
<|endoftext|> The result of ( 15 - 12 ) * 2 =
##########
With icd:   90 Without icd:   - Correct:   six
<|endoftext|> The result of ( 6 - 3 ) * 3 = nine. The result of ( 15 - 14 ) * 6 =
<|endoftext|> The result of ( 15 - 14 ) * 6 =
##########
With icd:   fourteen Without icd:   - Correct:   fourteen
<|endoftext|> The result of ( 11 - 10 ) * 15 = fifteen. The result of ( 11 - 10 ) * 14 =
<|endoftext|> The result of ( 11 - 10 ) * 14 =
##########
With icd:   three Without icd:   - Correct:   twelve
<|endoftext|> The result of ( 6 - 5 ) * 2 = two. The result of ( 19 - 15 ) * 3 =
<|endoftext|> The result of ( 19 - 15 ) * 3 =
##########
With icd:   twenty Without icd:   ( Co

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   four Without icd:   - Correct:   sixteen
<|endoftext|> The result of ( 12 - 10 ) * 2 = four. The result of ( 11 - 10 ) * 16 =
<|endoftext|> The result of ( 11 - 10 ) * 16 =
##########
With icd:   two Without icd:   - Correct:   four
<|endoftext|> The result of ( 7 - 6 ) * 11 = eleven. The result of ( 19 - 17 ) * 2 =
<|endoftext|> The result of ( 19 - 17 ) * 2 =
##########
With icd:   four Without icd:   - Correct:   six
<|endoftext|> The result of ( 15 - 13 ) * 2 = four. The result of ( 9 - 6 ) * 2 =
<|endoftext|> The result of ( 9 - 6 ) * 2 =
##########
With icd:   twenty Without icd:   - Correct:   twelve
<|endoftext|> The result of ( 16 - 14 ) * 9 = eighteen. The result of ( 13 - 9 ) * 3 =
<|endoftext|> The result of ( 13 - 9 ) * 3 =
##########
With icd:   sixteen Without icd:   ( Correct:   two
<|endoftext|> The result of ( 18 - 10 ) * 2 = sixteen. The result of ( 9 - 8 ) * 2 =
<|endoftext|> The result of ( 9 - 8 ) * 2 =
##########
With icd:   fifteen Without icd:   - C

In [84]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_only_results))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_only_results))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_only_results))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_only_results))

Accuracy with icd, numbers equality 0.0
Accuracy without icd, numbers equality 0.0
Accuracy with icd, tokens equality 0.580952380952381
Accuracy without icd, tokens equality 0.0


In [85]:
accuracy_base_only_results_words, accuracy_alt_only_results_words, accuracy_base_tok_only_results_words, accuracy_alt_tok_only_results_words = calc_accuracy(intervention_list_only_results_words, model)

  0%|          | 0/210 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   15 Without icd:   _____ Correct:  14
<|endoftext|>  four + four + six = 14.  three + five + six =
<|endoftext|>  three + five + six =
##########
With icd:   15 Without icd:   twenty Correct:  17
<|endoftext|>  three + two + eight = 13.  three + seven + seven =
<|endoftext|>  three + seven + seven =
##########
With icd:   12 Without icd:   ten Correct:  12
<|endoftext|>  four + two + eight = 14.  eight + two + two =
<|endoftext|>  eight + two + two =
##########
With icd:   14 Without icd:   ten Correct:  14
<|endoftext|>  ten + two + three = 15.  two + six + six =
<|endoftext|>  two + six + six =
##########
With icd:   20 Without icd:   twenty Correct:  19
<|endoftext|>  five + four + two = 11.  eight + six + five =
<|endoftext|>  eight + six + five =
##########
With icd:   11 Without icd:  
 Correct:  11
<|endoftext|>  nine + two + two = 13.  six + two + three =
<|endoftext|>  six + two + three =
##########
With icd:   19 Without icd:   twenty Correct:  17
<|endoftext|>  fo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   18 Without icd:   ? Correct:  18
<|endoftext|>  three + six + two = 11.  five + ten + three =
<|endoftext|>  five + ten + three =
##########
With icd:   18 Without icd:   twenty Correct:  18
<|endoftext|>  three + three + seven = 13.  four + nine + five =
<|endoftext|>  four + nine + five =
##########
With icd:   16 Without icd:   _____ Correct:  14
<|endoftext|>  five + five + seven = 17.  two + nine + three =
<|endoftext|>  two + nine + three =
##########
With icd:   17 Without icd:   seventeen Correct:  15
<|endoftext|>  nine + two + seven = 18.  two + ten + three =
<|endoftext|>  two + ten + three =
##########
With icd:   20 Without icd:   twenty Correct:  18
<|endoftext|>  seven + two + six = 15.  seven + four + seven =
<|endoftext|>  seven + four + seven =
##########
With icd:   14 Without icd:   ten Correct:  14
<|endoftext|>  four + eleven + two = 17.  two + four + eight =
<|endoftext|>  two + four + eight =
##########
With icd:   twelve Without icd:   twelve Correc

...

With icd:   40 Without icd:   - Correct:  4
<|endoftext|> The result of ( sixteen - thirteen ) * two = 6. The result of ( ten - nine ) * four =
<|endoftext|> The result of ( ten - nine ) * four =
##########
With icd:   2 Without icd:   - Correct:  2
<|endoftext|> The result of ( nine - eight ) * nine = 9. The result of ( thirteen - twelve ) * two =
<|endoftext|> The result of ( thirteen - twelve ) * two =
##########
With icd:   5 Without icd:   - Correct:  10
<|endoftext|> The result of ( ten - nine ) * eighteen = 18. The result of ( nineteen - seventeen ) * five =
<|endoftext|> The result of ( nineteen - seventeen ) * five =
##########
With icd:   - Without icd:   - Correct:  6
<|endoftext|> The result of ( eight - six ) * three = 6. The result of ( eight - seven ) * six =
<|endoftext|> The result of ( eight - seven ) * six =
##########
With icd:   18 Without icd:   ( Correct:  18
<|endoftext|> The result of ( sixteen - fourteen ) * seven = 14. The result of ( three - two ) * eighteen

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   26 Without icd:   - Correct:  12
<|endoftext|> The result of ( four - two ) * three = 6. The result of ( thirteen - seven ) * two =
<|endoftext|> The result of ( thirteen - seven ) * two =
##########
With icd:   2 Without icd:   - Correct:  6
<|endoftext|> The result of ( thirteen - twelve ) * eight = 8. The result of ( fifteen - twelve ) * two =
<|endoftext|> The result of ( fifteen - twelve ) * two =
##########
With icd:   - Without icd:   - Correct:  6
<|endoftext|> The result of ( six - three ) * three = 9. The result of ( fifteen - fourteen ) * six =
<|endoftext|> The result of ( fifteen - fourteen ) * six =
##########
With icd:   140 Without icd:   ( Correct:  14
<|endoftext|> The result of ( eleven - ten ) * fifteen = 15. The result of ( eleven - ten ) * fourteen =
<|endoftext|> The result of ( eleven - ten ) * fourteen =
##########
With icd:   3 Without icd:   - Correct:  12
<|endoftext|> The result of ( six - five ) * two = 2. The result of ( nineteen - fifteen ) *

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   0 Without icd:   - Correct:  16
<|endoftext|> The result of ( twelve - ten ) * two = 4. The result of ( eleven - ten ) * sixteen =
<|endoftext|> The result of ( eleven - ten ) * sixteen =
##########
With icd:   2 Without icd:   - Correct:  4
<|endoftext|> The result of ( seven - six ) * eleven = 11. The result of ( nineteen - seventeen ) * two =
<|endoftext|> The result of ( nineteen - seventeen ) * two =
##########
With icd:   6 Without icd:   - Correct:  6
<|endoftext|> The result of ( fifteen - thirteen ) * two = 4. The result of ( nine - six ) * two =
<|endoftext|> The result of ( nine - six ) * two =
##########
With icd:   27 Without icd:   - Correct:  12
<|endoftext|> The result of ( sixteen - fourteen ) * nine = 18. The result of ( thirteen - nine ) * three =
<|endoftext|> The result of ( thirteen - nine ) * three =
##########
With icd:   16 Without icd:   - Correct:  2
<|endoftext|> The result of ( eighteen - ten ) * two = 16. The result of ( nine - eight ) * two =


In [86]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_only_results_words))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_only_results_words))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_only_results_words))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_only_results_words))

Accuracy with icd, numbers equality 0.5380952380952381
Accuracy without icd, numbers equality 0.023809523809523808
Accuracy with icd, tokens equality 0.5380952380952381
Accuracy without icd, tokens equality 0.023809523809523808


In [89]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_only_results + accuracy_base_only_results_words))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_only_results + accuracy_alt_only_results_words))
''
print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_only_results + accuracy_base_tok_only_results_words))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_only_results + accuracy_alt_tok_only_results_words))

Accuracy with icd, numbers equality 0.26904761904761904
Accuracy without icd, numbers equality 0.011904761904761904
Accuracy with icd, tokens equality 0.5595238095238095
Accuracy without icd, tokens equality 0.011904761904761904


In [91]:
intervention_list_only_results_no_task_prompt = replace_numbers_with(intervention_list, conf, operands=False, result_few_shot=True, result_prompt=False, few_shot=True, prompt=False, replacement='words')

  0%|          | 0/210 [00:00<?, ?it/s]

In [92]:
intervention_list_only_results_no_task_prompt[0].__dict__

{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': '3 5 6',
 'operands_base': '3 5 6',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [1638],
 'res_base_tok': [1638],
 'res_string': None,
 'res_base_string': '14',
 'res_alt_string': '14',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': '3 + 5 + 6 =',
 'alt_string': '3 + 5 + 6 =',
 'few_shots': '4 + 4 + 6 = fourteen. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-12b-deduped-v0', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces

In [93]:
accuracy_base_only_results_no_task_prompt, accuracy_alt_only_results_no_task_prompt, accuracy_base_tok_only_results_no_task_prompt, accuracy_alt_tok_only_results_no_task_prompt = calc_accuracy(intervention_list_only_results_no_task_prompt, model)

  0%|          | 0/210 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   fourteen Without icd:   14 Correct:  14
<|endoftext|> 4 + 4 + 6 = fourteen. 3 + 5 + 6 =
<|endoftext|> 3 + 5 + 6 =
##########
With icd:   eighteen Without icd:   20 Correct:  17
<|endoftext|> 3 + 2 + 8 = thirteen. 3 + 7 + 7 =
<|endoftext|> 3 + 7 + 7 =
##########
With icd:   14 Without icd:   12 Correct:  12
<|endoftext|> 4 + 2 + 8 = fourteen. 8 + 2 + 2 =
<|endoftext|> 8 + 2 + 2 =
##########
With icd:   14 Without icd:   14 Correct:  14
<|endoftext|> 10 + 2 + 3 = fifteen. 2 + 6 + 6 =
<|endoftext|> 2 + 6 + 6 =
##########
With icd:   nineteen Without icd:   19 Correct:  19
<|endoftext|> 5 + 4 + 2 = eleven. 8 + 6 + 5 =
<|endoftext|> 8 + 6 + 5 =
##########
With icd:   thirteen Without icd:   11 Correct:  11
<|endoftext|> 9 + 2 + 2 = thirteen. 6 + 2 + 3 =
<|endoftext|> 6 + 2 + 3 =
##########
With icd:   seventeen Without icd:   17 Correct:  17
<|endoftext|> 4 + 6 + 8 = eighteen. 2 + 9 + 6 =
<|endoftext|> 2 + 9 + 6 =
##########


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   eighteen Without icd:   18 Correct:  18
<|endoftext|> 3 + 6 + 2 = eleven. 5 + 10 + 3 =
<|endoftext|> 5 + 10 + 3 =
##########
With icd:   nineteen Without icd:   18 Correct:  18
<|endoftext|> 3 + 3 + 7 = thirteen. 4 + 9 + 5 =
<|endoftext|> 4 + 9 + 5 =
##########
With icd:   seventeen Without icd:   14 Correct:  14
<|endoftext|> 5 + 5 + 7 = seventeen. 2 + 9 + 3 =
<|endoftext|> 2 + 9 + 3 =
##########
With icd:   17 Without icd:   15 Correct:  15
<|endoftext|> 9 + 2 + 7 = eighteen. 2 + 10 + 3 =
<|endoftext|> 2 + 10 + 3 =
##########
With icd:   eighteen Without icd:   20 Correct:  18
<|endoftext|> 7 + 2 + 6 = fifteen. 7 + 4 + 7 =
<|endoftext|> 7 + 4 + 7 =
##########
With icd:   sixteen Without icd:   12 Correct:  14
<|endoftext|> 4 + 11 + 2 = seventeen. 2 + 4 + 8 =
<|endoftext|> 2 + 4 + 8 =
##########
With icd:   twelve Without icd:   12 Correct:  12
<|endoftext|> 12 + 2 + 4 = eighteen. 5 + 5 + 2 =
<|endoftext|> 5 + 5 + 2 =
##########


...

With icd:   twenty Without icd:   - Correct:  4
<|endoftext|> The result of ( 16 - 13 ) * 2 = six. The result of ( 10 - 9 ) * 4 =
<|endoftext|> The result of ( 10 - 9 ) * 4 =
##########
With icd:   two Without icd:   - Correct:  2
<|endoftext|> The result of ( 9 - 8 ) * 9 = nine. The result of ( 13 - 12 ) * 2 =
<|endoftext|> The result of ( 13 - 12 ) * 2 =
##########
With icd:   five Without icd:   - Correct:  10
<|endoftext|> The result of ( 10 - 9 ) * 18 = eighteen. The result of ( 19 - 17 ) * 5 =
<|endoftext|> The result of ( 19 - 17 ) * 5 =
##########
With icd:   six Without icd:   - Correct:  6
<|endoftext|> The result of ( 8 - 6 ) * 3 = six. The result of ( 8 - 7 ) * 6 =
<|endoftext|> The result of ( 8 - 7 ) * 6 =
##########
With icd:   eighteen Without icd:   - Correct:  18
<|endoftext|> The result of ( 16 - 14 ) * 7 = fourteen. The result of ( 3 - 2 ) * 18 =
<|endoftext|> The result of ( 3 - 2 ) * 18 =
##########
With icd:   thirty Without icd:   - Correct:  12
<|endoftext|> Th

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad

With icd:   twenty Without icd:   - Correct:  12
<|endoftext|> The result of ( 4 - 2 ) * 3 = six. The result of ( 13 - 7 ) * 2 =
<|endoftext|> The result of ( 13 - 7 ) * 2 =
##########
With icd:   two Without icd:   - Correct:  6
<|endoftext|> The result of ( 13 - 12 ) * 8 = eight. The result of ( 15 - 12 ) * 2 =
<|endoftext|> The result of ( 15 - 12 ) * 2 =
##########
With icd:   90 Without icd:   - Correct:  6
<|endoftext|> The result of ( 6 - 3 ) * 3 = nine. The result of ( 15 - 14 ) * 6 =
<|endoftext|> The result of ( 15 - 14 ) * 6 =
##########
With icd:   fourteen Without icd:   - Correct:  14
<|endoftext|> The result of ( 11 - 10 ) * 15 = fifteen. The result of ( 11 - 10 ) * 14 =
<|endoftext|> The result of ( 11 - 10 ) * 14 =
##########
With icd:   three Without icd:   - Correct:  12
<|endoftext|> The result of ( 6 - 5 ) * 2 = two. The result of ( 19 - 15 ) * 3 =
<|endoftext|> The result of ( 19 - 15 ) * 3 =
##########
With icd:   twenty Without icd:   ( Correct:  16
<|endoftext|

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad

With icd:   four Without icd:   - Correct:  16
<|endoftext|> The result of ( 12 - 10 ) * 2 = four. The result of ( 11 - 10 ) * 16 =
<|endoftext|> The result of ( 11 - 10 ) * 16 =
##########
With icd:   two Without icd:   - Correct:  4
<|endoftext|> The result of ( 7 - 6 ) * 11 = eleven. The result of ( 19 - 17 ) * 2 =
<|endoftext|> The result of ( 19 - 17 ) * 2 =
##########
With icd:   four Without icd:   - Correct:  6
<|endoftext|> The result of ( 15 - 13 ) * 2 = four. The result of ( 9 - 6 ) * 2 =
<|endoftext|> The result of ( 9 - 6 ) * 2 =
##########
With icd:   twenty Without icd:   - Correct:  12
<|endoftext|> The result of ( 16 - 14 ) * 9 = eighteen. The result of ( 13 - 9 ) * 3 =
<|endoftext|> The result of ( 13 - 9 ) * 3 =
##########
With icd:   sixteen Without icd:   ( Correct:  2
<|endoftext|> The result of ( 18 - 10 ) * 2 = sixteen. The result of ( 9 - 8 ) * 2 =
<|endoftext|> The result of ( 9 - 8 ) * 2 =
##########
With icd:   fifteen Without icd:   - Correct:  18
<|endofte

In [94]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_only_results_no_task_prompt))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_only_results_no_task_prompt))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_only_results_no_task_prompt))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_only_results_no_task_prompt))

Accuracy with icd, numbers equality 0.06190476190476191
Accuracy without icd, numbers equality 0.5666666666666667
Accuracy with icd, tokens equality 0.06190476190476191
Accuracy without icd, tokens equality 0.5666666666666667


In [95]:
intervention_list_only_results_no_task_prompt_words = replace_numbers_with(intervention_list, conf, operands=True, result_few_shot=False, result_prompt=True, few_shot=True, prompt=True, replacement='words')

  0%|          | 0/210 [00:00<?, ?it/s]

In [96]:
intervention_list_only_results_no_task_prompt_words[0].__dict__

{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': '3 5 6',
 'operands_base': '3 5 6',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [1638],
 'res_base_tok': [25963],
 'res_string': None,
 'res_base_string': ' fourteen',
 'res_alt_string': '14',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'is_bloom': False,
 'is_mistral': False,
 'is_persimmon': False,
 'representation': 'arabic',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': ' three + five + six =',
 'alt_string': '3 + 5 + 6 =',
 'few_shots': ' four + four + six = 14. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-12b-deduped-v0', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_u

In [97]:
accuracy_base_only_results_no_task_prompt_words, accuracy_alt_only_results_no_task_prompt_words, accuracy_base_tok_only_results_no_task_prompt_words, accuracy_alt_tok_only_results_no_task_prompt_words = calc_accuracy(intervention_list_only_results_no_task_prompt_words, model)

  0%|          | 0/210 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   15 Without icd:   _____ Correct:   fourteen
<|endoftext|>  four + four + six = 14.  three + five + six =
<|endoftext|>  three + five + six =
##########
With icd:   15 Without icd:   twenty Correct:   seventeen
<|endoftext|>  three + two + eight = 13.  three + seven + seven =
<|endoftext|>  three + seven + seven =
##########
With icd:   12 Without icd:   ten Correct:   twelve
<|endoftext|>  four + two + eight = 14.  eight + two + two =
<|endoftext|>  eight + two + two =
##########
With icd:   14 Without icd:   ten Correct:   fourteen
<|endoftext|>  ten + two + three = 15.  two + six + six =
<|endoftext|>  two + six + six =
##########
With icd:   20 Without icd:   twenty Correct:   nineteen
<|endoftext|>  five + four + two = 11.  eight + six + five =
<|endoftext|>  eight + six + five =
##########
With icd:   11 Without icd:  
 Correct:   eleven
<|endoftext|>  nine + two + two = 13.  six + two + three =
<|endoftext|>  six + two + three =
##########
With icd:   19 Without icd: 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   18 Without icd:   ? Correct:   eighteen
<|endoftext|>  three + six + two = 11.  five + ten + three =
<|endoftext|>  five + ten + three =
##########
With icd:   18 Without icd:   twenty Correct:   eighteen
<|endoftext|>  three + three + seven = 13.  four + nine + five =
<|endoftext|>  four + nine + five =
##########
With icd:   16 Without icd:   _____ Correct:   fourteen
<|endoftext|>  five + five + seven = 17.  two + nine + three =
<|endoftext|>  two + nine + three =
##########
With icd:   17 Without icd:   seventeen Correct:   fifteen
<|endoftext|>  nine + two + seven = 18.  two + ten + three =
<|endoftext|>  two + ten + three =
##########
With icd:   20 Without icd:   twenty Correct:   eighteen
<|endoftext|>  seven + two + six = 15.  seven + four + seven =
<|endoftext|>  seven + four + seven =
##########
With icd:   14 Without icd:   ten Correct:   fourteen
<|endoftext|>  four + eleven + two = 17.  two + four + eight =
<|endoftext|>  two + four + eight =
##########
With i

...

With icd:   40 Without icd:   - Correct:   four
<|endoftext|> The result of ( sixteen - thirteen ) * two = 6. The result of ( ten - nine ) * four =
<|endoftext|> The result of ( ten - nine ) * four =
##########
With icd:   2 Without icd:   - Correct:   two
<|endoftext|> The result of ( nine - eight ) * nine = 9. The result of ( thirteen - twelve ) * two =
<|endoftext|> The result of ( thirteen - twelve ) * two =
##########
With icd:   5 Without icd:   - Correct:   ten
<|endoftext|> The result of ( ten - nine ) * eighteen = 18. The result of ( nineteen - seventeen ) * five =
<|endoftext|> The result of ( nineteen - seventeen ) * five =
##########
With icd:   - Without icd:   - Correct:   six
<|endoftext|> The result of ( eight - six ) * three = 6. The result of ( eight - seven ) * six =
<|endoftext|> The result of ( eight - seven ) * six =
##########
With icd:   18 Without icd:   ( Correct:   eighteen
<|endoftext|> The result of ( sixteen - fourteen ) * seven = 14. The result of ( three

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   26 Without icd:   - Correct:   twelve
<|endoftext|> The result of ( four - two ) * three = 6. The result of ( thirteen - seven ) * two =
<|endoftext|> The result of ( thirteen - seven ) * two =
##########
With icd:   2 Without icd:   - Correct:   six
<|endoftext|> The result of ( thirteen - twelve ) * eight = 8. The result of ( fifteen - twelve ) * two =
<|endoftext|> The result of ( fifteen - twelve ) * two =
##########
With icd:   - Without icd:   - Correct:   six
<|endoftext|> The result of ( six - three ) * three = 9. The result of ( fifteen - fourteen ) * six =
<|endoftext|> The result of ( fifteen - fourteen ) * six =
##########
With icd:   140 Without icd:   ( Correct:   fourteen
<|endoftext|> The result of ( eleven - ten ) * fifteen = 15. The result of ( eleven - ten ) * fourteen =
<|endoftext|> The result of ( eleven - ten ) * fourteen =
##########
With icd:   3 Without icd:   - Correct:   twelve
<|endoftext|> The result of ( six - five ) * two = 2. The result of (

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

With icd:   0 Without icd:   - Correct:   sixteen
<|endoftext|> The result of ( twelve - ten ) * two = 4. The result of ( eleven - ten ) * sixteen =
<|endoftext|> The result of ( eleven - ten ) * sixteen =
##########
With icd:   2 Without icd:   - Correct:   four
<|endoftext|> The result of ( seven - six ) * eleven = 11. The result of ( nineteen - seventeen ) * two =
<|endoftext|> The result of ( nineteen - seventeen ) * two =
##########
With icd:   6 Without icd:   - Correct:   six
<|endoftext|> The result of ( fifteen - thirteen ) * two = 4. The result of ( nine - six ) * two =
<|endoftext|> The result of ( nine - six ) * two =
##########
With icd:   27 Without icd:   - Correct:   twelve
<|endoftext|> The result of ( sixteen - fourteen ) * nine = 18. The result of ( thirteen - nine ) * three =
<|endoftext|> The result of ( thirteen - nine ) * three =
##########
With icd:   16 Without icd:   - Correct:   two
<|endoftext|> The result of ( eighteen - ten ) * two = 16. The result of ( ni

In [98]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_only_results_no_task_prompt_words))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_only_results_no_task_prompt_words))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_only_results_no_task_prompt_words))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_only_results_no_task_prompt_words))

Accuracy with icd, numbers equality 0.0
Accuracy without icd, numbers equality 0.0
Accuracy with icd, tokens equality 0.03333333333333333
Accuracy without icd, tokens equality 0.11428571428571428


In [101]:
print('Accuracy with icd, numbers equality', np.mean(accuracy_base_only_results_no_task_prompt + accuracy_base_only_results_no_task_prompt_words))
print('Accuracy without icd, numbers equality', np.mean(accuracy_alt_only_results_no_task_prompt + accuracy_alt_only_results_no_task_prompt_words))

print('Accuracy with icd, tokens equality', np.mean(accuracy_base_tok_only_results_no_task_prompt + accuracy_base_tok_only_results_no_task_prompt_words))
print('Accuracy without icd, tokens equality', np.mean(accuracy_alt_tok_only_results_no_task_prompt + accuracy_alt_tok_only_results_no_task_prompt_words))

Accuracy with icd, numbers equality 0.030952380952380953
Accuracy without icd, numbers equality 0.2833333333333333
Accuracy with icd, tokens equality 0.047619047619047616
Accuracy without icd, tokens equality 0.3404761904761905


In [111]:
np.sum(np.array(accuracy_base_tok_only_results_no_task_prompt + accuracy_base_tok_only_results_no_task_prompt_words) & np.array(accuracy_base_tok + accuracy_base_tok_words)) / np.sum(accuracy_base_tok_words + accuracy_base_tok)

np.float64(0.06896551724137931)

In [112]:
np.sum(np.array(accuracy_base_tok_only_results + accuracy_base_tok_only_results_words) & np.array(accuracy_base_tok + accuracy_base_tok_words)) / np.sum(accuracy_base_tok_words + accuracy_base_tok)

np.float64(0.7758620689655172)

accuracy_base_only_results_no_task_prompt + accuracy_base_only_results_no_task_prompt_words